## Parse and Convert Audio Files
---

#### Import Libraries

In [3]:
import os
import pydub
import pandas as pd
from scipy.io import wavfile
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence
from pydub.playback import play
import time

#### Define ffmpeg and ffprobe Path for PyDub Converter

In [4]:
pydub.AudioSegment.converter = '/Users/YOUR-USER/anaconda3/bin/ffmpeg'
pydub.AudioSegment.ffmpeg = '/Users/YOUR-USER/anaconda3/bin/ffmpeg'
pydub.AudioSegment.ffprobe = '/Users/YOUR-USER/anaconda3/bin/ffprobe'

We used the pydub Python package to parse and segment our mp3 files. Pydub accepts mp3 files and has methods that detect silence, detect audio, and split the mp3 files based on silence. We utilized the last method to pull samples of audio from our data. There are two parameters we passed in as well -- min_silence_len and silence_thresh. These determine how long a pause needs to be in milliseconds before it’s registered as silence, and how loud something needs to be in dbFS before its registered as audio. We played with a few min_silence_len and silence_thresh values, but settled on 2.5 seconds and -16dbFS for our dataset. 

#### Parse Through Files and Convert to .wav

In [ ]:
# Starting time
t0 = time.time()

# Insert file path with mp3 downloads
file_path = '../testing/mp3_downloads/'

# Loop through all files in file_path
for n, file in enumerate(os.listdir(file_path)):
    
    # Starting time
    t1 = time.time()
    
    # If file extension is .mp3
    if file.endswith('.mp3'):
        
        # Define audio to be split
        sound_file = AudioSegment.from_mp3(file_path + file)
        
        # Print file n, lenght and avg loudness for each file
        print(f'Analyzing File: {n}')
        print(f'File Lenght: {len(sound_file) / 1000}s')
        print(f'File Average Loudness: {round(sound_file.dBFS, 2)} dBFS')
        
                
        # Split files into smaller chunks
        chunks = split_on_silence(sound_file,                           # sound_file to be split
                                  min_silence_len = 4_000,              # min_silence_len in ms
                                  silence_thresh = sound_file.dBFS - 2, # silence thresh based on avg loudness
                                  keep_silence = 100)                   # add 100ms of silence before and after
        
        # Loop through all chunks and select the ones that are at least 2 seconds and less than 60 seconds
        for i, chunk in enumerate(chunks):
            
            #  Export chunks to that are at least 5 seconds and less than 60 seconds to .wav
            if (len(chunk) > 5_000) and (len(chunk) < 60_000):
                chunk.export(f"./testing/wav_converted_files/{file}_{i}.wav", format="wav")
        
        # Starting time
        print(f'File {n} RunTime: {round(time.time() - t1, 2)}s')
        print(' ')
        
# Print total runtime
print(f'Total RunTime:{round(time.time() - t0, 2)}s')

Analyzing File: 0
File Lenght: 771.997s
File Average Loudness: -23.66 dBFS


**Code Adapted from:** [Mitchell Bohman, Nour Zahlan, and Masiur Abik](https://github.com/mchbmn/radio-to-location) and [Joseph Hopkins, Carol, Chiu, Anthony Chapman, Kwamae Delva](https://github.com/delvakwa/police_radio_to_mapping)